In [1]:
from txgnn import TxData, TxGNN, TxEval

Tx_data = TxData(data_folder_path = 'data/train_data')
Tx_data.prepare_split(split = 'complex_disease', seed = 42, no_kg = False)

/home/apakiman/Repo/merck_gds_explr/submodules/TxGNN/env-TxGNN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!


In [ ]:
Tx_GNN = TxGNN(data = Tx_data, 
              weight_bias_track = False,
              proj_name = 'TxGNN',
              exp_name = 'TxGNN'
              )

In [3]:
Tx_GNN = TxGNN(data = Tx_data, 
              weight_bias_track = False,
              proj_name = 'TxGNN',
              exp_name = 'TxGNN'
              )

# to load a pretrained model: 
#TxGNN.load_pretrained('./models/checkpoints_all_seeds/TxGNN_1_complex_disease')
Tx_GNN.load_pretrained('.//models_complex_disease_42_500_batch_32')

# TxGNN.model_initialize(n_hid = 100, 
#                       n_inp = 100, 
#                       n_out = 100, 
#                       proto = True,
#                       proto_num = 3,
#                       attention = False,
#                       sim_measure = 'all_nodes_profile',
#                       bert_measure = 'disease_name',
#                       agg_measure = 'rarity',
#                       num_walks = 200,
#                       walk_mode = 'bit',
#                       path_length = 2)

In [ ]:
## here we did not run this, since the output is too long to fit into the notebook
TxGNN.pretrain(n_epoch = 2, 
               learning_rate = 1e-3,
               batch_size = 1024, 
               train_print_per_n = 20)

In [4]:
## here as a demo, the n_epoch is set to 30. Change it to n_epoch = 500 when you use it
TxGNN.finetune(n_epoch = 30, 
               learning_rate = 5e-4,
               train_print_per_n = 5,
               valid_per_n = 20)

Epoch: 0 LR: 0.00050 Loss 0.7093, Train Micro AUROC 0.7255 Train Micro AUPRC 0.6939 Train Macro AUROC 0.7399 Train Macro AUPRC 0.7160
----- AUROC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.79693318670585
('drug', 'indication', 'disease'): 0.7445254441840434
('drug', 'off-label use', 'disease'): 0.6797676581936182
('disease', 'rev_contraindication', 'drug'): 0.6662929983756278
('disease', 'rev_indication', 'drug'): 0.8655796509958669
('disease', 'rev_off-label use', 'drug'): 0.6864597079502434
----- AUPRC Performance in Each Relation -----
('drug', 'contraindication', 'disease'): 0.7543528548497082
('drug', 'indication', 'disease'): 0.716510172030091
('drug', 'off-label use', 'disease'): 0.6600957058863378
('disease', 'rev_contraindication', 'drug'): 0.6414576500763218
('disease', 'rev_indication', 'drug'): 0.8349642987768287
('disease', 'rev_off-label use', 'drug'): 0.6885557028376577
----------------------------------------------
Validation.....
Epoc

In [2]:
#TxGNN.save_model('./models/checkpoints_all_seeds/TxGNN_2_complex_disease')
from txgnn import TxData, TxGNN, TxEval

Tx_GNN = TxGNN(data = Tx_data, 
              weight_bias_track = False,
              proj_name = 'TxGNN',
              exp_name = 'TxGNN',
              device='cpu'
              )


Tx_GNN.load_pretrained('.//models_complex_disease_42_500_batch_32')

In [3]:
Tx_GNN.train_graphmask(relation = 'indication',
                      learning_rate = 3e-4,
                      allowance = 0.005,
                      epochs_per_layer = 1, #3,
                      penalty_scaling = 1,
                      valid_per_n = 2)#0)

output = Tx_GNN.retrieve_save_gates('./model_ckpt')
Tx_GNN.save_graphmask_model('./graphmask_model_ckpt')

gate_hidden_size:  32
Enabling layer 1


: 

In [13]:
from txgnn import TxEval
TxEval = TxEval(model = TxGNN)

load_pretrained_graphmask()

In [15]:
# evaluate individual diseases
result = TxEval.eval_disease_centric(disease_idxs = [12661.0, 11318.0], 
                                     relation = 'indication', 
                                     save_result = False)

# evaluate the entire test set
result = TxEval.eval_disease_centric(disease_idxs = 'test_set', 
                                     show_plot = False, 
                                     verbose = True, 
                                     save_result = True,
                                     return_raw = False)

  0%|          | 0/2 [00:00<?, ?it/s]

Evaluating relation: contraindication


  0%|          | 0/54 [00:00<?, ?it/s]

---------
AUROC mean:  0.752327526138064
AUROC std:  0.1858856686489882
---------
---------
AUPRC mean:  0.13968016130970864
AUPRC std:  0.2302530155722306
---------
---------
Accuracy mean:  0.8823494912172664
Accuracy std:  0.0646862634452154
---------
---------
Sensitivity mean:  0.4061223240656242
Sensitivity std:  0.39641674378976877
---------
---------
Specificity mean:  0.896884633314133
Specificity std:  0.06615214214692447
---------
---------
F1 mean:  0.11826961006777822
F1 std:  0.1974920901665274
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9817090124506684
NPV std:  0.03551809172552505
---------
---------
FPR mean:  0.10311536668586684
FPR std:  0.06615214214692448
---------
---------
FNR mean:  0.5938776759343759
FNR std:  0.39641674378976877
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  32.01851851851852
# of Pos std:  55.05165612631155
---------
---------
Recall@1% mean:  0.12043450669812626
Rec

  0%|          | 0/66 [00:00<?, ?it/s]

---------
AUROC mean:  0.8257557356962596
AUROC std:  0.16985070951055026
---------
---------
AUPRC mean:  0.07869411551085655
AUPRC std:  0.12907962935380474
---------
---------
Accuracy mean:  0.9232244712533441
Accuracy std:  0.045589373587365895
---------
---------
Sensitivity mean:  0.4268625714941528
Sensitivity std:  0.42846127191825706
---------
---------
Specificity mean:  0.9253084621815284
Specificity std:  0.04604332812527355
---------
---------
F1 mean:  0.0390161088191038
F1 std:  0.0557114334640408
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9974256498725271
NPV std:  0.005621376482176378
---------
---------
FPR mean:  0.07469153781847164
FPR std:  0.04604332812527355
---------
---------
FNR mean:  0.5731374285058471
FNR std:  0.42846127191825706
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  7.5
# of Pos std:  12.41608194677737
---------
---------
Recall@1% mean:  0.15907967319709976
Recall@1% s

  0%|          | 0/35 [00:00<?, ?it/s]

---------
AUROC mean:  0.8403037698819223
AUROC std:  0.1511260204653165
---------
---------
AUPRC mean:  0.13293952998553243
AUPRC std:  0.2118786815594834
---------
---------
Accuracy mean:  0.907187222715173
Accuracy std:  0.066338464223857
---------
---------
Sensitivity mean:  0.49357142857142855
Sensitivity std:  0.450846143266147
---------
---------
Specificity mean:  0.9095522375161197
Specificity std:  0.06736574584064527
---------
---------
F1 mean:  0.07526814486574862
F1 std:  0.12204437231852404
---------
---------
PPV mean:  nan
PPV std:  nan
---------
---------
NPV mean:  0.9967035858737601
NPV std:  0.004315405206175934
---------
---------
FPR mean:  0.09044776248388027
FPR std:  0.06736574584064525
---------
---------
FNR mean:  0.5064285714285715
FNR std:  0.450846143266147
---------
---------
FDR mean:  nan
FDR std:  nan
---------
---------
# of Pos mean:  3.3142857142857145
# of Pos std:  2.4351172288757157
---------
---------
Recall@1% mean:  0.08
Recall@1% std:  0

In [16]:
TxEval.retrieve_disease_idxs_test_set('indication')

array([ 7701., 12661., 11195., 16569.,  6343.,  1479., 12621., 14416.,
       13557., 16501., 13446., 13610.,  7105.,  9716.,  5822., 12320.,
       10023., 14513., 12929.,  5683.,  9950., 14661.,  8395.,  7990.,
        6031.,  6255., 14409.,  9589., 11885.,  4285.,  6377., 13130.,
        7773.,  9741., 13381., 15355.,  4536., 12891.,  9087., 14873.,
        6185.,  1812., 15301.,  6474.,  3381.,  6599., 16965.,  7880.,
       16698.,  6252.,  5764.,  4192.,  4467., 13107., 14501., 11318.,
        3452., 11349., 15911., 15100.,  1417.,  4152.,  1319.,  2220.,
       15989.,  8973.])